# Image Classification: More Pets!
Below we do the following:

1. Setup training environment.
2. Load images of cats, dogs, and rabbits.
3. Train an image classifier model.
4. Convert the model to CoreML format and download it.

## Environment Setup
All we need for this model is the turicreate library. Because the data is so small, we won't leverage a GPU for training here. If you wish to do that, chaneg this runtime type to GPU, make sure you have CUDA 10 and mxnet-cu100 installed, and then call `tc.config.set_num_gpus(-1)` prior to training.

In [0]:
# Install turicreate
!pip install turicreate

## Data Preparation and Model Training

In [0]:
# Import necessary libraries
import urllib
import tarfile

import coremltools
import turicreate as tc

In [0]:
# Specify the data set download url
data_url = "https://s3.amazonaws.com/skafos.example.data/ImageClassifier/MorePetImages.tar.gz"
data_path = "MorePetImages.tar.gz"

# Pull the compressed data and extract it
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [0]:
# Load images 
data = tc.image_analysis.load_images('MorePetImages', with_path=True, ignore_failure=True)

# From the path-name, create a label column. This labels each image as either a dog, cat, or rabbit 
data['label'] = data['path'].apply(lambda path: 'dog' if '/Dog' in path else ('cat' if '/Cat' in path else 'rabbit'))

In [0]:
# Make a train-set split
train_data, test_data = data.random_split(0.8)

In [0]:
train_data.head()

In [0]:
# Train an image classification model - consider increasing max_iterations
model = tc.image_classifier.create(
    dataset=train_data,
    target='label',
    model='resnet-50',
    batch_size=4,
    max_iterations=10
)

# Image Classification Training Docs:
# https://apple.github.io/turicreate/docs/api/generated/turicreate.image_classifier.create.html#turicreate.image_classifier.create

In [0]:
# Let's see how we do on the hold out data
predictions = model.classify(test_data)
predictions['ground_truth'] = test_data['label']
predictions

## Model Export and Download

In [0]:
# Specify the CoreML model name
model_name = 'ImageClassifier'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 

In [0]:
from google.colab import files
files.download(coreml_model_name)